Todo
* delete the iterated hyperparameters save it in pickel or json
* add concurrency- after first .fit start the next one + do the suff for the previous one

## Sample Input

In [21]:
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
import pandas as pd

### Classification

In [22]:
data=pd.read_csv('diabetes.csv')
X=data.drop("Outcome",axis=1)
y=data['Outcome']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

svc_para = {'kernel': ['sigmoid',"linear"], 'gamma': [1, 10]}
para = ParameterGrid(svc_para)

from sklearn.svm import SVC
model = SVC()

### Regression

In [23]:
# data =pd.read_csv('USA_Housing.csv')
# X=data.drop(["Address",'Price'], axis=1)
# y=data['Price']
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

# knn_reg_para = {"n_neighbors" : [1,2,3,4,5], "weights" : ["uniform", "distance"]}
# para = ParameterGrid(knn_reg_para)

# from sklearn.neighbors import KNeighborsRegressor
# model = KNeighborsRegressor()

In [24]:
from sklearn.model_selection import ParameterGrid
from sklearn.base import is_classifier, is_regressor
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
import pandas as pd
import sys,csv,os

class GridCheckpoint:

    def __init__(self,model, params, X_train, X_test, y_train, y_test):
        self.model = model
        self.params = params
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.save_file = 'history.csv'
        self.save_data = {'Hyperparameters' : None, 'Train Accuracy' : None, "Test Accuracy" : None, "Train Loss" : None, "Test Loss" : None}
        
    def save_log(self,data_arr):
        if not os.path.exists("history.csv") or os.stat("history.csv").st_size == 0: # create save file
            with open(self.save_file, 'w',newline='') as file: # newline to avoid space
                writer = csv.DictWriter(file, fieldnames=list(data_arr.keys()))
                writer.writeheader()
                file.close()

        with open(self.save_file, 'a') as file:
            writer = csv.writer(file)
            writer.writerow(data_arr.values())
            file.close()
                 
    def run(self):
        for p in self.params:
            self.save_data[list(self.save_data)[0]] = str(p)
            print("{} - {}\n".format(list(self.save_data)[0], self.save_data[list(self.save_data)[0]]))
            
            if is_classifier(self.model):
                if not(self.model.get_params()["probability"]):  # probability=false in SVC causes error in predict_proba in self.score if is_classifier
                    model = self.model.__class__(**p,probability=True).fit(self.X_train,self.y_train) # must use **kwargs when using ParameterGrid
            else:
                model = self.model.__class__(**p).fit(self.X_train,self.y_train)
                
            self.score(model)
            print("\t------------------------------------")
            
    def score(self,fitted_model):
        score_vals = []

        score_vals.append(fitted_model.score(self.X_train,self.y_train))
        score_vals.append(fitted_model.score(self.X_test,self.y_test))
        
        if is_classifier(self.model): # For classifier
            score_vals.append(log_loss(self.y_train, fitted_model.predict_proba(X_train)))
            score_vals.append(log_loss(self.y_test, fitted_model.predict_proba(X_test)))
        
        if is_regressor(self.model): # For Regressor
            score_vals.append(mean_squared_error(self.y_train, fitted_model.predict(X_train)))
            score_vals.append(mean_squared_error(self.y_test, fitted_model.predict(X_test)))  

        for i in range(len(self.save_data)):
            if i == 0:
                # print("{} - {}".format(list(self.save_data)[i], self.save_data[list(self.save_data)[i]])) # show before to see which hyper takes times
                pass
            else:
                self.save_data[list(self.save_data)[i]] = score_vals[i-1]
                print("{} | {}".format(list(self.save_data)[i], self.save_data[list(self.save_data)[i]]))

        self.save_log(self.save_data)

    def best_params(self, tolerate_overfit = 0.10): # percentage
        df = pd.read_csv(self.save_file)
        score_mean = []
        score_std = []
        found_best = False

        for i in range(len(df)):
            score_mean.append(df[[list(self.save_data)[1],list(self.save_data)[2]]].iloc[i].mean())
            score_std.append(df[[list(self.save_data)[1],list(self.save_data)[2]]].iloc[i].std())

        for i in range(len(score_mean)):
            index = score_mean.index(sorted(score_mean, reverse=True)[i])
            print(index)
            if score_std[index] <= (tolerate_overfit/2): # convert tolerate_overfit to percent, n/2% on each side of mean
                print(df.iloc[index])
                found_best = True
                break
    
        if not found_best:
            print("All model overfits over {}%".format(tolerate_overfit*100))

    def display_logs(self):
        if not os.path.exists("history.csv") or os.stat("history.csv").st_size == 0:
            print("No logs found")
        else:
            print(pd.read_csv(self.save_file))

In [25]:
grid = GridCheckpoint(model, para,X_train,X_test,y_train,y_test)

In [26]:
grid.run()

Hyperparameters - {'gamma': 1, 'kernel': 'sigmoid'}

Train Accuracy | 0.6499068901303539
Test Accuracy | 0.6536796536796536
Train Loss | 0.6475158662194473
Test Loss | 0.6452200115486163
	------------------------------------
Hyperparameters - {'gamma': 1, 'kernel': 'linear'}

Train Accuracy | 0.7802607076350093
Test Accuracy | 0.7445887445887446
Train Loss | 0.46424656438735207
Test Loss | 0.5082577588989724
	------------------------------------
Hyperparameters - {'gamma': 10, 'kernel': 'sigmoid'}

Train Accuracy | 0.6499068901303539
Test Accuracy | 0.6536796536796536
Train Loss | 0.6475158662194473
Test Loss | 0.6452200115486163
	------------------------------------
Hyperparameters - {'gamma': 10, 'kernel': 'linear'}

Train Accuracy | 0.7802607076350093
Test Accuracy | 0.7445887445887446
Train Loss | 0.46141876728790804
Test Loss | 0.5114425278424826
	------------------------------------


In [27]:
grid.best_params()

1
Hyperparameters    {'gamma': 1, 'kernel': 'linear'}
Train Accuracy                             0.780261
Test Accuracy                              0.744589
Train Loss                                 0.464247
Test Loss                                  0.508258
Name: 1, dtype: object
